In [1]:
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers>=4.55.3" \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00
Using Python 3.12.11 environment at: /usr
Resolved 18 packages in 51ms
Prepared 2 packages in 785ms
Uninstalled 2 packages in 518ms
Installed 2 packages in 70ms
 - tokenizers==0.22.0
 + tokenizers==0.21.4
 - transformers==4.56.1
 + transformers==4.55.4
Using Python 3.12.11 environment at: /usr
Resolved 1 package in 7ms
Prepared 1 package in 36ms
Uninstalled 1 package in 3ms
Installed 1 package in 14ms
 - trl==0.23.0
 + trl==0.22.2


In [2]:
import unsloth
import torch
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = None,
    max_seq_length = 1024,
    load_in_4bit = True,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.9.4: Fast Gpt_Oss patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.
Unsloth: Gpt_Oss does not support SDPA - switching to fast eager.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [5]:
model.print_trainable_parameters()

trainable params: 3,981,312 || all params: 20,918,738,496 || trainable%: 0.0190


In [6]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-09-12

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>Equation: x^5 + 3x^4 - 10 = 3 => x^5 + 3x^4 -13=0. Solve for real roots. Maybe factor? try integer roots: try x=1:1+3-13=-9 no. x=2


In [7]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "medium",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-09-12

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The user asks: "Solve x^5 + 3x^4 - 10 = 3." Probably solve for x? The equation is x^5 + 3x^4 - 10 = 3. Move 3 to left: x^5 + 3x^4


In [8]:
from transformers import TextStreamer

messages = [
    {"role": "user", "content": "Solve x^5 + 3x^4 - 10 = 3."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "high",
).to(model.device)

_ = model.generate(**inputs, max_new_tokens = 64, streamer = TextStreamer(tokenizer))

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-09-12

Reasoning: high

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>user<|message|>Solve x^5 + 3x^4 - 10 = 3.<|end|><|start|>assistant<|channel|>analysis<|message|>The prompt: "Solve x^5 + 3x^4 - 10 = 3." Solve the equation. So we set x^5 + 3x^4 - 10 = 3. That simplifies: x^5 + 3x^4 - 10 = 


In [9]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.29M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages'],
    num_rows: 1000
})

In [10]:
dataset

Dataset({
    features: ['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages'],
    num_rows: 1000
})

In [11]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages', 'text'],
    num_rows: 1000
})

In [15]:
dataset['text'][0]

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-09-12\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nreasoning language: French\n\nYou are an AI chatbot with a lively and energetic personality.<|end|><|start|>user<|message|>Can you show me the latest trends on Twitter right now?<|end|><|start|>assistant<|channel|>analysis<|message|>D\'accord, l\'utilisateur demande les tendances Twitter les plus récentes. Tout d\'abord, je dois vérifier si j\'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l\'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.\n\nJe devr

In [16]:
from trl import SFTConfig, SFTTrainer

In [17]:
args = SFTConfig(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 50,
    learning_rate = 2e-4,
    lr_scheduler_type = "linear",
    logging_steps = 1,
    optim = 'paged_adamw_32bit',
    weight_decay=0.01,
    seed=3407,
    output_dir='outputs',
)

In [18]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = args,
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith24 (abeshith24-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.126400
2,2.897100
3,2.367600
4,2.132200
5,1.958500
6,2.086800
7,1.771300
8,1.643800
9,1.843900
10,1.676700


TrainOutput(global_step=50, training_loss=1.4576924657821655, metrics={'train_runtime': 1510.3259, 'train_samples_per_second': 0.132, 'train_steps_per_second': 0.033, 'total_flos': 2.146300212245299e+16, 'train_loss': 1.4576924657821655, 'epoch': 0.2})

In [21]:
FastLanguageModel.for_inference(model)

PeftModel(
  (base_model): LoraModel(
    (model): GptOssForCausalLM(
      (model): GptOssModel(
        (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
        (layers): ModuleList(
          (0-23): 24 x GptOssDecoderLayer(
            (self_attn): GptOssAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2880, out_features=4096, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2880, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [22]:
def test_model(prompt_text, reasoning_effort="medium", max_tokens=512):
    messages = [{"role": "user", "content": prompt_text}]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
        reasoning_effort=reasoning_effort,
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the new generated text
    prompt_text_full = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
    generated_text = response[len(prompt_text_full):].strip()

    return generated_text

In [23]:
print("🧮 MATHEMATICAL REASONING TEST:")
math_result = test_model("Solve x^5 + 3x^4 - 10 = 3.", reasoning_effort="high", max_tokens=256)
print(f"Result: {math_result}")
print("-" * 60)

🧮 MATHEMATICAL REASONING TEST:
Result: analysisWe need to solve the equation x^5 + 3x^4 - 10 = 3. Let's rewrite it in standard polynomial form:

x^5 + 3x^4 - 10 = 3  
=> x^5 + 3x^4 - 10 - 3 = 0  
=> x^5 + 3x^4 - 13 = 0

So the equation is x^5 + 3x^4 - 13 = 0.

We need to find the real roots of this polynomial. Let's start by checking for rational roots using the Rational Root Theorem. The possible rational roots are factors of 13 divided by factors of 1 (since the leading coefficient is 1). So the possible rational roots are ±1, ±13.

Testing x = 1:  
1^5 + 3*1^4 - 13 = 1 + 3 - 13 = -9 ≠ 0

Testing x = -1:  
(-1)^5 + 3*(-1)^4 - 13 = -1 + 3 - 13 = -11 ≠ 0

Testing x = 13:  
13^5 + 3*
------------------------------------------------------------


In [24]:
print("🎯 DIFFERENT REASONING EFFORTS:")
test_prompt = "Explain the concept of machine learning and give 3 practical examples."

# Low effort
low_result = test_model(test_prompt, reasoning_effort="low", max_tokens=200)
print(f"LOW EFFORT: {low_result}")
print("-" * 30)

# Medium effort
med_result = test_model(test_prompt, reasoning_effort="medium", max_tokens=300)
print(f"MEDIUM EFFORT: {med_result}")
print("-" * 30)

# High effort
high_result = test_model(test_prompt, reasoning_effort="high", max_tokens=400)
print(f"HIGH EFFORT: {high_result}")
print("-" * 60)


🎯 DIFFERENT REASONING EFFORTS:
LOW EFFORT: analysisI need to explain machine learning, define it, and give examples.assistantcommentary to=functions{"name":"explain_concept","arguments":{"concept":"machine_learning"}}commentarycommentaryI will call the function to gather the necessary information.assistantcommentary to=functions to=functions write_concept{"name":"explain_concept","arguments":{"concept":"machine_learning"}}commentary to=functions commentarycommentary to=functions write_concept{"name":"generate_examples","arguments":{"concept":"machine_learning","count":3}}commentary to=functions commentary to=functions commentary to=functions commentary to=functions commentary to=functions commentary to=functions commentary to=functions commentary to=functions commentary to=functions commentary
------------------------------
MEDIUM EFFORT: analysisThe user wants an explanation of the concept of machine learning and 3 practical examples. I need to define machine learning first, then give

In [25]:
test_queries = [
    "What are the main differences between supervised and unsupervised learning?",
    "Write a Python function to calculate fibonacci numbers recursively.",
    "Explain quantum computing in simple terms.",
    "How do neural networks learn from data?",
    "What is the time complexity of binary search?"
]

print("🔬 COMPREHENSIVE MODEL TESTING:")
for i, query in enumerate(test_queries, 1):
    result = test_model(query, reasoning_effort="medium", max_tokens=250)
    print(f"\nTest {i}: {query}")
    print(f"Response: {result}")
    print("-" * 40)

🔬 COMPREHENSIVE MODEL TESTING:

Test 1: What are the main differences between supervised and unsupervised learning?
Response: analysisI need to explain the differences between supervised and unsupervised learning. First, recall that supervised learning involves labeled data and predicting outputs, while unsupervised learning uses unlabeled data to find patterns. I should mention examples for each, like regression and classification for supervised, and clustering and dimensionality reduction for unsupervised. Also, discuss their differences in terms of objectives and learning methods. It's important to highlight that supervised learning requires labeled data and optimizes a model to predict labels, whereas unsupervised learning doesn't have labels and focuses on discovering inherent structures. I should also touch on methods like decision trees for supervised and k-means for unsupervised. Think about how to structure this explanation: start with definitions, then objectives, methods, ex